In [1]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
import pandas as pd
from kafka import KafkaProducer
import json
from time import time

# Define the topic name
topic_name = "green-trips"

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',  # Change this if your Kafka is running elsewhere
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Read the CSV file
csv_file = "green_tripdata_2019-10.csv"
df = pd.read_csv(csv_file)

# Select only the required columns
df = df[[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]]

# Start timing
t0 = time()

# Send each row as a message
for _, row in df.iterrows():
    message = row.to_dict()
    producer.send(topic_name, value=message)

# Flush the producer
producer.flush()

# End timing
t1 = time()
print(f"Data sending took {t1 - t0:.2f} seconds")


/var/folders/vd/dl48s8js5_z0k6_rb5jv19xw0000gn/T/ipykernel_7439/2416842485.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Data sending took 21.81 seconds


In [5]:
from kafka import KafkaConsumer
import json

# Define topic name
topic_name = "green-trips"

# Initialize Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers="localhost:9092",  # Ensure this matches your Kafka broker
    auto_offset_reset="earliest",  # Start from the beginning if no offset is stored
    enable_auto_commit=True,  # Automatically commit offsets
    value_deserializer=lambda x: json.loads(x.decode("utf-8")),  # Deserialize JSON messages
)

print(f"Consuming the first 5 messages from topic: {topic_name}")

# Counter to limit the number of messages
message_count = 0
max_messages = 5

# Start consuming messages
for message in consumer:
    if message_count >= max_messages:
        break  # Stop after 5 messages

    print(f"Received message: {message.value}")
    message_count += 1

print(f"Consumed {message_count} messages.")


Consuming the first 5 messages from topic: green-trips
Received message: {'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}
Received message: {'lpep_pickup_datetime': '2019-10-01 00:18:11', 'lpep_dropoff_datetime': '2019-10-01 00:22:38', 'PULocationID': 43, 'DOLocationID': 263, 'passenger_count': 1.0, 'trip_distance': 0.8, 'tip_amount': 0.0}
Received message: {'lpep_pickup_datetime': '2019-10-01 00:09:31', 'lpep_dropoff_datetime': '2019-10-01 00:24:47', 'PULocationID': 255, 'DOLocationID': 228, 'passenger_count': 2.0, 'trip_distance': 7.5, 'tip_amount': 0.0}
Received message: {'lpep_pickup_datetime': '2019-10-01 00:37:40', 'lpep_dropoff_datetime': '2019-10-01 00:41:49', 'PULocationID': 181, 'DOLocationID': 181, 'passenger_count': 1.0, 'trip_distance': 0.9, 'tip_amount': 0.0}
Received message: {'lpep_pickup_datetime': '2019-10-01 00:08: